In [10]:
import os 
import requests
from bs4 import BeautifulSoup
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pull in variables for scripts

In [19]:
# get the read key
with open("../src/api_credentials.json") as f:
    purple_air_creds = json.load(f)["purple_air_credentials"]

# api read key created at https://develop.purpleair.com/keys
api_read_key = purple_air_creds["purple_air_read_key"]

In [9]:
# get the coordinates

with open("../src/coordinates.json") as f:
    sensor_coordinates = json.load(f)['sensor_box']

NWlat = sensor_coordinates['sensor_NWlat']
NWlng = sensor_coordinates['sensor_NWlng']
SElat = sensor_coordinates['sensor_SElat']
SElng = sensor_coordinates['sensor_SElng']

print(NWlat)
print(NWlng)
print(SElat)
print(SElng)

41.325
-112.13
40.548
-111.806


# Pull all purple air sensors in the sensor map area and store in a .json file
## DO THIS ONCE

In [17]:
# done, commented out below to avoid running
#headers= {'X-API-Key': api_read_key}

#outdoor_url = 'https://api.purpleair.com/v1/sensors?fields=pm10.0&location_type=0&nwlng=' + str(NWlng) + '&nwlat='+ str(NWlat) + '&selng=' + str(SElng) + '&selat=' + str(SElat)

#outdoor_response = requests.get(outdoor_url, headers=headers)
#print("Status Code: ", outdoor_response.status_code)

#outdoor_content = json.loads(outdoor_response.content)
#outdoor_data = outdoor_content["data"]
#outdoor_columns = outdoor_content["fields"]

#outdoor_df = pd.DataFrame(outdoor_data, columns=outdoor_columns)

# Extract the first column (sensor_index)
#outdoor_sensor_list = outdoor_df.iloc[:, 0].tolist()

# Create a dictionary with the desired format
#outdoor_json_data = {"outdoor_sensor_list": outdoor_sensor_list}

Status Code:  200


# Define function to pull data from a sensor using its index and only the columns desired

In [23]:
import argparse
import json
import pandas as pd
import requests

def pull_current_outdoor_purpleair_sensor_data(fields):
    """
    Retrieve the latest sensor data from the PurpleAir API for multiple sensors.

    Args:
    - fields (list of str): A list of fields to retrieve for each sensor.

    Returns:
    - Pandas dataframe: Datafrane containing the specified fields for each sensor.
    """

    # pull in the outdoor sensors from .json file
    with open("../src/purpleair_parameters.json") as outdoor_sensors:
        #outdoor_sensor_list = json.load(outdoor_sensors)["outdoor_sensor_list"]
        outdoor_sensor_list = json.load(outdoor_sensors)["sensor_test"]

    # get the read key from .json file
    with open("../src/api_credentials.json") as f:
        purple_air_creds = json.load(f)["purple_air_credentials"]
    # store it in a variable
    api_read_key = purple_air_creds["purple_air_read_key"]
    
    # join the list of fields into a comma-separated string,
    # which is the required input for the API call
    fields_str = ",".join(fields)

    # initiate empty list to hold data for all sensor_ids identified
    list_of_sensor_data = []
    
    for sensor_id in outdoor_sensor_list:
        # construct the URL for the API request
        url = f"https://api.purpleair.com/v1/sensors/{sensor_id}"
        
        # set the query parameters for the request  
        params = {
            "fields": fields_str
        }

        # set the headers including the API key
        headers = {
            "X-API-Key": api_read_key
        }

        # send the GET request to the PurpleAir API
        sensor_response = requests.get(url, params=params, headers=headers)

        # parse the JSON response
        sensor_response_json = sensor_response.json()

        # extract the sensor data from the response
        sensor_data = sensor_response_json.get('sensor')

        # append to the list
        list_of_sensor_data.append(sensor_data)

    # convert the list to a dataframe
    current_sensor_data_df = pd.DataFrame(list_of_sensor_data)

    # return the dataframe
    return current_sensor_data_df

if __name__ == "__main__":
    # create argument parser
    parser = argparse.ArgumentParser(description='Fetch PurpleAir Sensor Data')

    # add command-line arguments
    parser.add_argument('--fields', type=int, help='Fields')
    parser.add_argument('--output_file', type=str, help='Output file name')

    # parse command-line arguments
    args = parser.parse_args()

    # call fetch_and_combine_data function with provided arguments
    annual_data = fetch_and_combine_data(args.start_year, args.end_year, aqs_email, aqs_pw, args.param, minlat, maxlat, minlng, maxlng)
    
    # Save data to CSV if output_file argument is provided
    if args.output_file:
        annual_data.to_csv(args.output_file, index=False)
        print(f"Data saved to {args.output_file}")
    else:
        # If output_file argument is not provided, print the data
        print(annual_data)

def main():
    # create argument parser
    parser = argparse.ArgumentParser(description='Fetch sensor data from PurpleAir API.')

    # add command-line arguments
    parser.add_argument('--fields', nargs='+', default=['name', 'latitude', 'longitude', 'pm2.5'], help='List of fields to retrieve for each sensor')
    parser.add_argument('--output_file', type=str, help='Output file name')

    # parse command-line arguments
    args = parser.parse_args()

    # call pull_current_outdoor_purpleair_sensor_data function with provided arguments
    sensor_data = pull_current_outdoor_purpleair_sensor_data(args.fields)

    # save data to CSV if output_file argument is provided
    if args.output_file:
        sensor_data.to_csv(args.output_file, index=False)
        print(f"Data saved to {args.output_file}")
    else:
        print(sensor_data)

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--fields FIELDS]
                             [--output_file OUTPUT_FILE]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/jaredfeldman/Library/Jupyter/runtime/kernel-a2415422-a9d9-485a-8b3e-33f2ce9c63b6.json


SystemExit: 2

/Users/jaredfeldman/miniforge3/envs/capstone/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
# Pull it without the argparse part

In [31]:
def pull_current_outdoor_purpleair_sensor_data(fields):
    """
    Retrieve the latest sensor data from the PurpleAir API for multiple sensors.

    Args:
    - fields (list of str): A list of fields to retrieve for each sensor.

    Returns:
    - Pandas dataframe: Datafrane containing the specified fields for each sensor.
    """

    # pull in the outdoor sensors from .json file
    with open("../src/purpleair_parameters.json") as outdoor_sensors:
        outdoor_sensor_list = json.load(outdoor_sensors)["sensor_test"]

    # get the read key from .json file
    with open("../src/api_credentials.json") as f:
        purple_air_creds = json.load(f)["purple_air_credentials"]
    # store it in a variable
    api_read_key = purple_air_creds["purple_air_read_key"]
    
    # join the list of fields into a comma-separated string,
    # which is the required input for the API call
    fields_str = ",".join(fields)

    # initiate empty list to hold data for all sensor_ids identified
    list_of_sensor_data = []
    
    for sensor_id in outdoor_sensor_list:
        # construct the URL for the API request
        url = f"https://api.purpleair.com/v1/sensors/{sensor_id}"
        
        # set the query parameters for the request  
        params = {
            "fields": fields_str
        }

        # set the headers including the API key
        headers = {
            "X-API-Key": api_read_key
        }

        # send the GET request to the PurpleAir API
        sensor_response = requests.get(url, params=params, headers=headers)

        # parse the JSON response
        sensor_response_json = sensor_response.json()

        # extract the sensor data from the response
        sensor_data = sensor_response_json.get('sensor')

        # append to the list
        list_of_sensor_data.append(sensor_data)

    # convert the list to a dataframe
    current_sensor_data_df = pd.DataFrame(list_of_sensor_data)

    # return the dataframe
    return current_sensor_data_df

In [32]:
#z = pull_current_outdoor_purpleair_sensor_data(["pm10.0"])
z

,sensor_index,pm10.0
0,77,18.5
1,443,7.8
2,525,0.0
3,984,1.0
4,992,12.9
...,...,...
217,204009,2.0
218,204315,1.1
219,207743,2.7
220,208783,0.7


In [34]:
762470 - 761804

666